In [1]:
import pandas as pd
import xlrd
import numpy as np
from collections import Counter
import os
import xlsxwriter
import time



In [2]:
pd.options.mode.chained_assignment = None

In [7]:
def dropTopLeftRight(spec):

    # finding elements from varsNamesSet
    start_norm = time.time()
    spec = spec.replace('\s+', ' ', regex=True).astype(str).apply(lambda x: x.str.lower())
    spec = spec.replace('nan', np.nan)
    end_norm = time.time()
#     print('Таблица нормализована за: {}'.format(end_norm-start_norm))
    normTime = (end_norm-start_norm)

    start_norm = time.time()
    specTrueFalseMap = spec.isin(varsNamesSet).replace(False, np.nan)
    if specTrueFalseMap.count(axis='columns').sum() == 0:
#         print('Не удалось обработать файл')
        return  pd.DataFrame()


    # находим строку
    heading_row = specTrueFalseMap.count(axis='columns').idxmax()+2
    # находим левую границу
    a = spec.iloc[specTrueFalseMap.count(axis='columns').idxmax()].isnull().replace(True,1)
    if 'Unnamed: ' in a.idxmin():
        heading_left_end = int(a.idxmin().split('Unnamed: ')[1])+1
    else:
        heading_left_end = a.index.get_loc(a.idxmin())+1
    # находим правую границу
    min_a = min(list(a))
    len_a = len(list(a))
    if [i for i, j in enumerate(list(a)) if j == min_a][-1] < len_a-1:
        heading_right_end = [i for i, j in enumerate(list(a)) if j == min_a][-1]+1
    else:
        heading_right_end = len_a

    RC_title = ((heading_row,heading_left_end),(heading_row,heading_right_end))

    # находим строку с номерами столбцов
    # в doesContainNumbersRow тупл с bool и координатами(тоже тупл)
    doesContainNumbersRow = (False,())

    try:
        if int(spec.iloc[heading_row-1][1])+1 == int(spec.iloc[heading_row-1][2]):
            doesContainNumbersRow = (True,((heading_row+1,heading_left_end),(heading_row+1,heading_right_end)))

    except:
        pass

    if not doesContainNumbersRow[0]:
        try:
            if int(spec.iloc[heading_row][1])+1 == int(spec.iloc[heading_row][2]):
                doesContainNumbersRow = (True,((heading_row+2,heading_left_end),(heading_row+2,heading_right_end)))
        except:
            pass

    # находим количество заголовков
    numberOfTitleNames = (~spec.iloc[heading_row-2].isnull()).replace(True,1).sum()

    # находим повторяющиеся заголовки и их количество
    titelNames = Counter(spec.iloc[heading_row-2])
    repeatedTitleNames = Counter(el for el in titelNames.elements() if titelNames[el] >= 2)
#     print(repeatedTitleNames)
    amountOFRepeatedTitleNames = sum(repeatedTitleNames.values())


    # выясняем двойной ли заголовок
    titleIsDoubled = False

    if doesContainNumbersRow[0]:
        if heading_row == doesContainNumbersRow[1][0][0]-2:
            titleIsDoubled = True
    else:
        smallHead = list(spec.iloc[heading_row-1:].head(3).count(axis=1))
        if smallHead[0] < sum(smallHead)/len(smallHead):
            titleIsDoubled = True

    # находим вторую (английскую) строку заголовков
    secondTitle = (False,())

    if titleIsDoubled:
        try:
            if (~spec.iloc[heading_row-4].isnull()).replace(True,1).sum() > 3:
                secondTitle = (True, ((heading_row-2,heading_left_end),(heading_row-2,heading_right_end)))
        except:
            pass
    else:
        try:
            if (~spec.iloc[heading_row-3].isnull()).replace(True,1).sum() > 3:
                secondTitle = (True, ((heading_row-1,heading_left_end),(heading_row-1,heading_right_end)))
        except:
            pass


    # # дропаем строку с номерами
    # if doesContainNumbersRow[0]:
    #     if doesContainNumbersRow[1][0][0] - heading_row == 1:
    #         spec.drop(heading_row+1, inplace = True)
    #     elif doesContainNumbersRow[1][0][0] - heading_row == 1:
    #         spec.drop(heading_row, inplace = True)

    spec = spec.iloc[specTrueFalseMap.count(axis='columns').idxmax():].dropna(how='all')
    # print(spec)
    listOfEmptyTitleFields = [i for i, j in spec.iloc[0].isnull().replace(False,np.nan).to_dict().items() if j == 1]
    spec = spec.drop(columns=listOfEmptyTitleFields)
    spec.columns = spec.iloc[0]
    spec = spec.iloc[1:]
    spec.reset_index(inplace = True,drop=True)
    spec.columns.name = ''

    end_norm = time.time()
#     print('Чтение диапазона заголовков выполнено за: {}'.format(end_norm-start_norm))
    titleReadingTime = end_norm-start_norm
    return spec, RC_title, numberOfTitleNames, repeatedTitleNames, amountOFRepeatedTitleNames, titleIsDoubled, secondTitle,doesContainNumbersRow, titleReadingTime, normTime

def renameColumnsNames(spec):
    start_norm = time.time()
    changedColumnsNames = []
    changedColumnsNamesDict = {}

    for i,col in enumerate(spec):
        if col in varsNamesSet:
            for j in varsDF:
                if col in list(varsDF[j]):
                    spec.rename(columns={col: j}, inplace=True)
                    changedColumnsNames.append(j)
                    try:
                        if col in list(copy_columns[j]):
                            changedColumnsNamesDict[col] = ((j,'copy_columns'))
                    except:
                        pass

                    try:
                        if col in list(text_columns[j]):
                            changedColumnsNamesDict[col] = ((j,'text_columns'))
                    except:
                        pass

                    try:
                        if col in list(eng_vars_columns[j]):
                            changedColumnsNamesDict[col] = ((j,'eng_vars_columns'))
                    except:
                        pass

                    break
        else:
            changedColumnsNamesDict[col] = (('не обработан',''))



    # numberOfColumnsAppearences = Counter(changedColumnsNames)
    changedColumnsNames = list(set(changedColumnsNames))
    end_norm = time.time()
#     print('Обработка диапазона заголовков рабочей таблицы выполнена за: {}'.format(end_norm-start_norm))
    return spec, changedColumnsNames, changedColumnsNamesDict

def dropSpecsBottom(spec,titleIsDoubled,doesContainNumbersRow):
    apearencesOfNanInRows = spec.isnull().sum(axis=1)

    # h = Counter(apearencesOfNanInRows.head(int(len(apearencesOfNanInRows)/2))).most_common()

    # listOfRowIndicesToDelete = list(apearencesOfNanInRows.loc[apearencesOfNanInRows > 1 + h[0][0]].index)


    if doesContainNumbersRow[0] &  titleIsDoubled:
        apearenceOfNanInRows = spec.iloc[2].isnull().sum()
        # spec.drop([0,1], inplace=True)
    elif doesContainNumbersRow[0] | titleIsDoubled:
        apearenceOfNanInRows = spec.iloc[1].isnull().sum()
        # spec.drop(0, inplace=True)
    else:
        apearenceOfNanInRows = spec.iloc[0].isnull().sum()

    # apearenceOfNanInRows = spec.iloc[0].isnull().sum()
    listOfRowIndicesToDelete = list(apearencesOfNanInRows.loc[apearencesOfNanInRows > 1 + apearenceOfNanInRows].index)

    spec.drop(listOfRowIndicesToDelete,inplace=True)

    if doesContainNumbersRow[0] &  titleIsDoubled:
        # apearenceOfNanInRows = spec.iloc[2].isnull().sum()
        try:
            spec.drop([0,1], inplace=True)
        except:
            pass
    elif doesContainNumbersRow[0] | titleIsDoubled:
        # apearenceOfNanInRows = spec.iloc[1].isnull().sum()
        try:
            spec.drop(0, inplace=True)
        except:
            pass

    return spec

def findSameColumnNames(spec):
    # если есть одноименные столбцы мы их мержим. в astype можно указать другой тип данных
    # к сожалению этот код меняет порядок в столбцов
    def sjoin(x): return ';'.join(x[x.notnull()].astype(str))
    spec = spec.groupby(level=0, axis=1).apply(lambda x: x.apply(sjoin, axis=1))
    return spec

def changingColumnsValues(spec):
    columnsData = {}

    for (columnName, columnData) in spec.iteritems():
        columnsData[columnName] = {}
        columnsData[columnName]['amountOfEmptyFields'] = spec[columnName].isnull().sum()
        columnsData[columnName]['amountOfChangedFields'] = 0
        columnsData[columnName]['amountOfUnchangedFields'] = len(columnData)
        columnsData[columnName]['unchangedFields'] = set()

        if columnName in list(copy_columns.columns):
            columnsData[columnName]['amountOfChangedFields'] = columnData.astype(str).str.contains(',', regex=False).replace(np.nan, False).sum()
            columnsData[columnName]['amountOfUnchangedFields'] = len(columnData) - columnsData[columnName]['amountOfChangedFields']
            # print(spec[columnName].loc[~(spec[columnName].astype(str).str.contains(',', regex=False).replace(np.nan, True))])
            # print(set(spec[columnName].loc[~(spec[columnName].astype(str).str.contains(',', regex=False).replace(np.nan, True))]))
            columnsData[columnName]['unchangedFields'] = set(spec[columnName].loc[~(spec[columnName].astype(str).str.contains(',', regex=False).replace(np.nan, True))])
            spec[columnName] = spec[columnName].astype(str).str.replace(',','.')

        elif columnName in list(eng_vars_columns.columns):
    #         for val in columnData:
            for k in eng_vars_Connection_columns.keys():
                columnsData[columnName]['unchangedFields'] = columnsData[columnName]['unchangedFields'].union(set(spec[columnName].loc[~spec[columnName].astype(str).str.contains(k, regex=False).replace(np.nan, True)]))
                columnsData[columnName]['amountOfChangedFields'] += len(spec[columnName].loc[spec[columnName].astype(str).str.contains(k, regex=False).replace(np.nan, True)])
                columnsData[columnName]['amountOfUnchangedFields'] -= len(spec[columnName].loc[spec[columnName].astype(str).str.contains(k, regex=False).replace(np.nan, True)])
                spec[columnName] = spec[columnName].astype(str).str.replace(k,str(eng_vars_Connection_columns[k][0]),regex=False)
            for k in eng_vars_Material_columns.keys():
                columnsData[columnName]['unchangedFields'] = columnsData[columnName]['unchangedFields'].union(set(spec[columnName].loc[~spec[columnName].astype(str).str.contains(k, regex=False).replace(np.nan, True)]))
                columnsData[columnName]['amountOfChangedFields'] += len(spec[columnName].loc[spec[columnName].astype(str).str.contains(k, regex=False).replace(np.nan, True)])
                columnsData[columnName]['amountOfUnchangedFields'] -= len(spec[columnName].loc[spec[columnName].astype(str).str.contains(k, regex=False).replace(np.nan, True)])
                spec[columnName] = spec[columnName].astype(str).str.replace(k,str(eng_vars_Material_columns[k][0]),regex=False)
            for k in eng_vars_SeismoCat_columns.keys():
                columnsData[columnName]['unchangedFields'] = columnsData[columnName]['unchangedFields'].union(set(spec[columnName].loc[~spec[columnName].astype(str).str.contains(k, regex=False).replace(np.nan, True)]))
                columnsData[columnName]['amountOfChangedFields'] += len(spec[columnName].loc[spec[columnName].astype(str).str.contains(k, regex=False).replace(np.nan, True)])
                columnsData[columnName]['amountOfUnchangedFields'] -= len(spec[columnName].loc[spec[columnName].astype(str).str.contains(k, regex=False).replace(np.nan, True)])
                spec[columnName] = spec[columnName].astype(str).str.replace(k,str(eng_vars_SeismoCat_columns[k][0]),regex=False)

            columnsData[columnName]['unchangedFields'] = list(set(columnsData[columnName]['unchangedFields']))
        # print('{}: {} : {}'.format(columnName, columnsData[columnName]['unchangedFields'], len(columnsData[columnName]['unchangedFields']) ))
    return spec, columnsData

def findingTable(spec):
    stats = {}
    # droppig unwanted columns and rows from top and left
    spec, stats['RC_title'], stats['numberOfTitleNames'], stats['repeatedTitleNames'], stats['amountOFRepeatedTitleNames'], stats['titleIsDoubled'], stats['secondTitle'],stats['doesContainNumbersRow'],stats['titleReadingTime'], stats['normTime'] = dropTopLeftRight(spec)

    # changing columns names
    spec, changedColumnsNames, stats['changedColumnsNamesDict'] = renameColumnsNames(spec)
    stats['changedColumnsNamesAmount'] = len(changedColumnsNames)


    # finding the bottom of spec
    start_read = time.time()
    spec = dropSpecsBottom(spec,stats['titleIsDoubled'],stats['doesContainNumbersRow'])

    # находим диапазон значений
    stats['RC_rows'] = ( ( stats['RC_title'][0][0] + 1 , stats['RC_title'][0][1] ) , ( stats['RC_title'][0][0] + 1 , stats['RC_title'][1][1] ) )

    if stats['doesContainNumbersRow'][0] &  stats['titleIsDoubled']:
        stats['RC_rows'] = ((stats['RC_title'][0][0]+3,stats['RC_title'][0][1]),(spec.tail(1).index[0] + stats['RC_title'][0][0] + 1,stats['RC_title'][1][1]))
    elif stats['doesContainNumbersRow'][0] | stats['titleIsDoubled']:
        if stats['doesContainNumbersRow'][0]:
            stats['RC_rows'] = ((stats['RC_title'][0][0]+2,stats['RC_title'][0][1]),(spec.tail(1).index[0] + stats['RC_title'][0][0] + 1,stats['RC_title'][1][1]))
        else:
            stats['RC_rows'] = ((stats['RC_title'][0][0]+2,stats['RC_title'][0][1]),(spec.tail(1).index[0] + stats['RC_title'][0][0] + 1,stats['RC_title'][1][1]))


    stats['amountOfRows'] = stats['RC_rows'][1][0] - stats['RC_rows'][0][0] + 1

    end_read = time.time()
#     print('Чтение диапазона характеристик выполнено за: {}'.format(end_read-start_read))
    stats['chReadingTime'] = end_read-start_read

    spec = spec.fillna('REPLACEMENT')

    # find and merge columns with same name
    spec = findSameColumnNames(spec)
    copiesFreqDF = pd.DataFrame(spec.value_counts(subset=changedColumnsNames))

    spec['countClones'] = 0
    for i,r in enumerate(spec.iterrows()):
        spec['countClones'].iloc[i] = copiesFreqDF.loc[tuple(r[1].loc[changedColumnsNames])][0]
        copiesFreqDF.loc[tuple(r[1].loc[changedColumnsNames])][0] = 0

    spec = spec.loc[spec['countClones']!=0]

    if 'DN' in spec.columns:
        spec = spec.sort_values(by=['DN'])

    spec = spec.replace('REPLACEMENT', np.nan)

    spec , stats['columnsStats']= changingColumnsValues(spec)
    return spec, stats

In [8]:
xls = pd.ExcelFile('vars_for_columns.xlsx')

copy_columns = pd.read_excel(xls, 'copy_columns').replace('\s+', ' ', regex=True).astype(str).apply(lambda x: x.str.lower())
text_columns = pd.read_excel(xls, 'text_columns').replace('\s+', ' ', regex=True).astype(str).apply(lambda x: x.str.lower())
eng_vars_columns = pd.read_excel(xls, 'eng_vars_columns').replace('\s+', ' ', regex=True).astype(str).apply(lambda x: x.str.lower())

eng_vars_Connection_columns = pd.read_excel('english_vars_Connection.xlsx').replace('\s+', ' ', regex=True)
eng_vars_Connection_columns.connection_ru = eng_vars_Connection_columns.connection_ru.str.lower()
eng_vars_Connection_columns = eng_vars_Connection_columns[['connection_ru','connection']]
eng_vars_Connection_columns = eng_vars_Connection_columns.set_index('connection_ru').T.to_dict('list')

eng_vars_Material_columns = pd.read_excel('english_vars_Material.xlsx').replace('\s+', ' ', regex=True)
eng_vars_Material_columns.material_ru = eng_vars_Material_columns.material_ru.str.lower()
eng_vars_Material_columns = eng_vars_Material_columns[['material_ru','material']]
eng_vars_Material_columns = eng_vars_Material_columns.set_index('material_ru').T.to_dict('list')

eng_vars_SeismoCat_columns = pd.read_excel('english_vars_SeismoCat.xlsx').replace('\s+', ' ', regex=True).astype(str).apply(lambda x: x.str.lower())
eng_vars_SeismoCat_columns = eng_vars_SeismoCat_columns[['SeismoCat_ru','SeismoCat']]
eng_vars_SeismoCat_columns = eng_vars_SeismoCat_columns.set_index('SeismoCat_ru').T.to_dict('list')

varsDF = pd.concat([copy_columns,text_columns,eng_vars_columns],axis=1)

varsNamesSet = set(copy_columns.stack().tolist())|set(text_columns.stack().tolist())|set(eng_vars_columns.stack().tolist())
varsNamesSet.discard('nan')

<ipython-input-8-fe15245abe2a>:10: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  eng_vars_Connection_columns = eng_vars_Connection_columns.set_index('connection_ru').T.to_dict('list')
<ipython-input-8-fe15245abe2a>:15: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  eng_vars_Material_columns = eng_vars_Material_columns.set_index('material_ru').T.to_dict('list')
<ipython-input-8-fe15245abe2a>:19: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  eng_vars_SeismoCat_columns = eng_vars_SeismoCat_columns.set_index('SeismoCat_ru').T.to_dict('list')


In [9]:
unchangedFields = {}
titleNames = {}

In [10]:
PATH = '100specs'

for entry in os.listdir(PATH):
    if os.path.isfile(os.path.join(PATH, entry)):
        if (entry.split('.')[-1] == 'xls' )|(entry.split('.')[-1] == 'xlsx' ):
            spec = pd.read_excel(os.path.join(PATH, entry))
            spec, stats = findingTable(spec)
            for stat in stats['columnsStats']:
                if stat in unchangedFields.keys():
                    unchangedFields[stat] = unchangedFields[stat].union(stats['columnsStats'][stat]['unchangedFields'])
                else:
                    unchangedFields[stat] = set(stats['columnsStats'][stat]['unchangedFields'])
                
            for t in stats['changedColumnsNamesDict']:
                if t not in titleNames.keys():
                    titleNames[t] = stats['changedColumnsNamesDict'][t]
                        
                    
            

In [11]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('table_'+PATH.split('/')[-1]+'.xlsx', engine='xlsxwriter')


# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet2 = workbook.add_worksheet()
worksheet3 = workbook.add_worksheet()


row = 2
for key in titleNames.keys():

    worksheet2.write('A' + str(row), key)
    worksheet2.write('B' + str(row), str(titleNames[key]))
    row = row + 1


col = 2

listOfUnchangedValues = set()

def isNaN(num):
    return num != num

for key in unchangedFields.keys():
    worksheet3.write(2, col, key)
    worksheet3.write(3, col, 'unchangedFields')
    k = 0
    for el in unchangedFields[key]:
        if not isNaN(el):
            worksheet3.write(3+k, col+1,el)
            k += 1

    col += 2


# print(listOfUnchangedValues)
# Close the Pandas Excel writer and output the Excel file.
writer.save()